In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import preprocessor
import gensim
import json
import preprocessing
import re
import string
import pandas as pd
import functions as f
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# gensim
from gensim import corpora, models, similarities, matutils
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
corpus = f.read_json('/media/nick/Comms/RC_2018-10.json')
corpus

In [ ]:
corpus.columns

In [ ]:
deleted_rows = corpus[corpus['body'] == '[deleted]'].index
corpus.drop(deleted_rows, inplace=True)

In [ ]:
comments = corpus.iloc[:, 1]
comments = [str(comment) for comment in comments]

In [ ]:
comments = [comment.lower() for comment in comments]
comments = [x.split('http', 1)[0] for x in comments]
comments = [x.split('pic.',1)[0] for x in comments]
comments = [re.sub('[%s]' % re.escape(string.punctuation), '', text)  for text in comments]
comments = [comment.replace('\r', "") for comment in comments]
comments = [comment.replace('\n', "") for comment in comments]
comments

In [ ]:
# # Create a CountVectorizer for parsing/counting words
# count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
#                                    stop_words='english', 
#                                    strip_accents='unicode', 
#                                    token_pattern="\\b[a-z][a-z]+\\b", 
#                                    lowercase=True,
#                                   )

# comms = count_vectorizer.fit(comments)

vectorizer = TfidfVectorizer()
comms = vectorizer.fit_transform(comments).todense()
print(vectorizer.get_feature_names())
print(X.shape)

In [ ]:
comms

In [ ]:
# Create the term-document matrix
# Transpose it so the terms are the rows
doc_word = count_vectorizer.transform(comments).transpose()

In [ ]:
doc_word[1]

In [ ]:
df = pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names())
df = df.loc[df.index.drop_duplicates()]

In [ ]:
df

In [ ]:
doc_word.shape

In [ ]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)

In [ ]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
len(id2word)

In [ ]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaMulticore(corpus=corpus, num_topics=14, id2word=id2word, passes=5)

In [ ]:
lda.print_topics()

In [ ]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [ ]:
# Store the documents' topic vectors in a list so we can take a peek
lda_docs = [doc for doc in lda_corpus]

In [ ]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:5]

In [ ]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(2)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = count_vectorizer.get_feature_names())
topic_word